<a href="https://colab.research.google.com/github/takumiaaa/ML-code/blob/master/MNIST_1_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
from torchvision import datasets, transforms

# データ拡張として標準化とTensor化を行う．
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5, ), (0.5, ))])
ds_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
ds_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoaderを作成する。
batch_size = 128  # バッチサイズ
#　　batch_size　メモリが小さい環境の場合，この値を小さくする
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_test  = torch.utils.data.DataLoader(ds_test,  batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 非常にシンプルな4層のMLPを設計する。
# nn.Sequential()に積層していく
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(SimpleCNN, self).__init__()
        self.in_channels = in_channels
        self.output_dim = output_dim
        self.encoder = nn.Sequential(
# そのままだと画像が小さくなり続けるのでpadding=2としてサイズ低下を防止する。
            nn.Conv2d(in_channels, 32, 5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(64, 128, 5, padding=2),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(128, output_dim)
        )

    # サイズが（1x1）になるようにAvgPoolingする
    def forward(self, x):
        x = self.encoder(x)
        x = F.adaptive_avg_pool2d(x, (1, 1)) 
        x = x.view(-1, 128)
        return self.classifier(x)

# SimpleCNNは(1→32→64→128)と変形させるネットワークである。
# インスタンス化　実体化
net = SimpleCNN(1, 10) # 入力(1)→出力(10)
net

SimpleCNN(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [5]:
def get_default_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

# GPUを使用する。
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = get_default_device()

# SimpleCNNは(1→32→64→128)と変形させるネットワークである。
cnn = SimpleCNN(1, 10) # 入力(1)→出力(10)

# 出力を試しに確認してみる．
for x,y in dl_test: # 1イテレーション分のデータを取得
    break
net = net.to("cpu")
out = net(x)
print("Xの書式", x.shape)
print("Shapeの確認", out.shape)
print("1データ目の出力", out[0])
transforms.functional.to_pil_image(x[0].view(28, 28))  # 画像の確認


Xの書式 torch.Size([128, 1, 28, 28])
Shapeの確認 torch.Size([128, 10])
1データ目の出力 tensor([-0.0314, -0.0718, -0.0236, -0.0783, -0.0096, -0.0495,  0.0119,  0.0100,
        -0.0048,  0.1082], grad_fn=<SelectBackward0>)


In [6]:
# 損失関数：Pytorch標準のbinary cross entropy lossを採用。
criterion = nn.CrossEntropyLoss()

# 最適化手法：Adamをlr=0.001で利用する．
lr = 0.001
opt = torch.optim.Adam(cnn.parameters(), lr=lr) # cnnのパラメータを入れる．

In [7]:
# 1エポック実行
def train(net, opt, criterion, loader, device="cpu"):
    net.train() #NETを訓練モードにする
    running_loss = 0.0
    correct = 0
    # loader内のデータを1周訓練させる
    for i, (inputs, labels) in enumerate(loader, 0):
        opt.zero_grad()  # 勾配をゼロにする．
        inputs, labels = inputs.to(device), labels.to(device)
        
        out = net(inputs)   # Feed forward
        loss = criterion(out, labels)  # Calculate Loss
        loss.backward()     # Backward
        opt.step()    # Update weights
        
        correct += sum(out.argmax(dim=1)==labels)
        running_loss += loss.item()
    train_acc = (correct/len(loader.dataset)).cpu().item()
    return running_loss, train_acc

In [8]:
# 評価
def evaluate(net, loader, device="cpu"):
    net.eval()  # netを評価モードにする．

    with torch.no_grad():  # 評価なので勾配計算を不要とする．
        Xs, ys, preds = [], [], []
        for inputs, labels in loader:
            Xs.append(inputs)
            ys.append(labels.detach().numpy())
            preds.append(net(inputs.to(device)).to("cpu").detach().numpy())

    import numpy as np
    Xs = torch.cat(Xs, dim=0)
    ys = np.concatenate(ys, axis=0)
    preds = np.concatenate(preds, axis=0).argmax(axis=1)
    acc = (ys==preds).mean()
    return acc, (Xs, ys, preds)

In [9]:
from tqdm.notebook import trange
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

hist_cnn = []
n_epochs = 20  # 20 epochs訓練する

cnn = cnn.to(device)  # GPUで訓練できるようにする．
cnn.train()  # cnnを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(cnn, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(cnn, dl_test, device)
    hist_cnn.append([loss, train_acc, test_acc])

epoch:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def plot_history(history):
    fig, ax1 = plt.subplots()

    # 損失の推移を描画する。
    ax1.plot(history[:, 0], label="train loss", linestyle="dashed", marker = 'o')
    ax1.set_xlabel("Epoch")
    
    ax2 = ax1.twinx()
    ax2.plot(history[:, 1], label="train acc", marker = 'o')
    ax2.plot(history[:, 2], label="test acc", marker = 'o')

    fig.legend(loc="center")
    plt.show()

plot_history(np.array(hist))
print(hist[-1])